In [0]:
file_path = "/FileStore/tables/yellow_tripdata_2020_01.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         1|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0.0|                  0.3

In [0]:
#Query 1: Add a Column Named "Revenue"
from pyspark.sql.functions import col

df_with_revenue = df.withColumn(
    "Revenue",
    col("fare_amount") + col("extra") + col("mta_tax") +
    col("improvement_surcharge") + col("tip_amount") +
    col("tolls_amount") + col("total_amount")
)
df_with_revenue.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|           Revenue|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         1|                 N|         238|         239|           1|        6.0|  

In [0]:
#Query 2: Increasing Count of Total Passengers by Area
df.groupBy("PULocationID").sum("passenger_count").orderBy("sum(passenger_count)", ascending=False).show()

+------------+--------------------+
|PULocationID|sum(passenger_count)|
+------------+--------------------+
|         237|              433243|
|         161|              425986|
|         236|              403347|
|         230|              360096|
|         162|              351011|
|         186|              338952|
|         132|              326402|
|          48|              297148|
|         142|              294502|
|         170|              289593|
|         234|              284965|
|         163|              267047|
|         239|              263583|
|          79|              244515|
|         141|              237341|
|          68|              227635|
|         164|              218138|
|         107|              215684|
|         238|              199367|
|         138|              197352|
+------------+--------------------+
only showing top 20 rows



In [0]:
#Query 3: Realtime Average Fare/Total Earning Amount Earned by 2 Vendors
from pyspark.sql.functions import avg

df.groupBy("VendorID").agg(
    avg("fare_amount").alias("avg_fare"),
    avg("total_amount").alias("avg_total_earning")
).show(2)

+--------+------------------+------------------+
|VendorID|          avg_fare| avg_total_earning|
+--------+------------------+------------------+
|       1|12.231264768274743|18.113429405162332|
|       2|12.624907753075965|18.648347164088374|
+--------+------------------+------------------+
only showing top 2 rows



In [0]:
#Query 4: Moving Count of Payments Made by Each Payment Mode
df.groupBy("payment_type").count().orderBy("count", ascending=False).show()

+------------+-------+
|payment_type|  count|
+------------+-------+
|           1|4694897|
|           2|1593834|
|        null|  65441|
|           3|  32770|
|           4|  18065|
|           5|      1|
+------------+-------+



In [0]:
#Query 5: Highest Two Gaining Vendors on a Particular Date
from pyspark.sql.functions import to_date, col, sum

date = '2020-01-01'
df_filtered_date = df.withColumn("pickup_date", to_date("tpep_pickup_datetime")).filter(col("pickup_date") == date)
df_filtered_date.groupBy("VendorID").agg(
    sum("fare_amount").alias("total_earnings")
).orderBy("total_earnings", ascending=False).show(2)


+--------+-----------------+
|VendorID|   total_earnings|
+--------+-----------------+
|       2|1648923.260000001|
|       1|677241.2599999987|
+--------+-----------------+
only showing top 2 rows



In [0]:
#Query 6: Most Number of Passengers Between a Route of Two Locations
df.groupBy("PULocationID", "DOLocationID").sum("passenger_count").orderBy("sum(passenger_count)", ascending=False).show()

+------------+------------+--------------------+
|PULocationID|DOLocationID|sum(passenger_count)|
+------------+------------+--------------------+
|         237|         236|               67885|
|         236|         236|               57662|
|         236|         237|               56488|
|         237|         237|               49757|
|         264|         264|               44789|
|         239|         238|               30402|
|         239|         142|               28755|
|         161|         237|               27492|
|         142|         239|               27260|
|         186|         230|               25857|
|         141|         236|               25850|
|         238|         239|               25431|
|         237|         162|               25160|
|         230|         186|               24846|
|         237|         161|               24779|
|         186|         161|               23489|
|         237|         141|               22829|
|         263|      

In [0]:
#Query 7: Get Top Pickup Locations with Most Passengers in Last 5/10 Seconds
from pyspark.sql.functions import window, current_timestamp

# Add a current timestamp column for simulation purposes
df = df.withColumn("current_time", current_timestamp())

# Simulate a streaming query
df.groupBy(window("current_time", "10 seconds"), "PULocationID") \
    .sum("passenger_count") \
    .orderBy("sum(passenger_count)", ascending=False) \
    .show(10)

+--------------------+------------+--------------------+
|              window|PULocationID|sum(passenger_count)|
+--------------------+------------+--------------------+
|{2024-07-27 15:20...|         237|              433243|
|{2024-07-27 15:20...|         161|              425986|
|{2024-07-27 15:20...|         236|              403347|
|{2024-07-27 15:20...|         230|              360096|
|{2024-07-27 15:20...|         162|              351011|
|{2024-07-27 15:20...|         186|              338952|
|{2024-07-27 15:20...|         132|              326402|
|{2024-07-27 15:20...|          48|              297148|
|{2024-07-27 15:20...|         142|              294502|
|{2024-07-27 15:20...|         170|              289593|
+--------------------+------------+--------------------+
only showing top 10 rows

